In [2]:
import sys

sys.path.extend(['.', '..'])

import platform
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import pandas as pd

gpu_token = tf.test.is_gpu_available('GPU')

if platform.machine() == 'arm64':
    os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/manual_dataset")

from pycparser import c_parser, c_generator
import pycparser
from Utils.w2b_encoding_pt import *
from Utils.cparser_dataframe_gen import *


2024-04-23 02:53:17.550047: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-23 02:53:17.550085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
raw_code, comments = code_preprocessing('saksh_parser_simple.c') # remove comments
uvt, moded_code = find_undefined_types_in_variables(raw_code, gpu_token) # replacing undefined types in variables with 'int' 
uft, remoded_code = find_undefined_function_types(moded_code, gpu_token) # replacing undefined function types with 'int'

In [9]:
text = remoded_code.replace("  ","").split('\n')
text = " ".join(text)
text
# text=text.replace(" ","")
# text

'#include <stdio.h>  int main ( void ){  printf ( "Executing critical code...\\n" );  return 0 ;  }  '

In [10]:
# text = remoded_code[152:]
parser = c_parser.CParser()
generator = c_generator.CGenerator()
ast = parser.parse(text, filename='??.c')
explore = {pycparser.c_ast.If: {'node': (),'key': 'cond', 'branches': ('iftrue', 'iffalse')}, 
                       pycparser.c_ast.FuncCall: {'node': (),'key' : 'name' , 'branches': ()},
                       pycparser.c_ast.FuncDef: {'node': (),'key' : 'decl', 'branches': ('body',)},
                       pycparser.c_ast.FileAST: {'node': (),'key' : 'ext[0]', 'branches': ()}
                       } 
                    #    pycparser.c_ast.Switch: (),
                    #    pycparser.c_ast.For: ()}

If_explore = {pycparser.c_ast.BinaryOp : {'branches': ('left', 'right')}}

HAMMING_WEIGHT = 4

ParseError: ??.c:1:1: Directives not supported yet

In [7]:
ast

FileAST(ext=[
            ]
        )

In [19]:
# text = 'static void iv_plain64(guint8 *iv, guint iv_size, guint64 sector)'
text = 'guint8 iv[16] ;'
# variable_pattern = r'\b((?:[a-zA-Z_]\w*\**)\s+\**\s*\**[a-zA-Z_]\w*)\s*(?:,|\s*;|\s*=|\s*\))'
variable_pattern = r'\b((?:[a-zA-Z_]\w*\**)\s+\**\s*\**[a-zA-Z_]\w*\[*\w*\]*)\s*(?:,|\s*;|\s*=|\s*\))'

matches = re.findall(variable_pattern, text)
l = [i.split()[0] for i in matches]
for i in matches:
    print(i.split()[0].replace(i.split()[0], "int ") + i.split()[1])
    

int iv[16]


In [8]:
print(remoded_code)

#include <stdio.h> 
int main ( void )  {     
  printf ( "Executing critical code...\n" )  ; 
  return 0 ; 
}

              
